In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt

In [ ]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

In [ ]:
def vv(row):
    X = row.split('-')
    return X[0]+X[1]+X[2]

In [ ]:
def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d

In [ ]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n = path + '\Cleaned_Stocks_Holders_1399-07-21_From96.csv'
df = pd.read_csv(n)

In [ ]:
df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)

In [ ]:
HolderData = df

In [ ]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n1 = path + '\Stocks_Prices_1399-07-20' + '.csv'
df1 = pd.read_csv(n1)

In [ ]:
df1['jalaliDate'] = df1['jalaliDate'].apply(vv)
df = df1
symbols = [ 'سپرده','هما','وهنر-پذيره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['name'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)
df = df.drop(df[(df.name == 'اتکای')&(df.close_price == 1000)].index)
df = df.drop_duplicates()
df = df.drop(df.loc[(df['volume'] == 0)].index).sort_values(by =['name','jalaliDate']).drop(columns = ['volume','quantity']).rename(columns = {'name':'symbol'})
df = DriveYearMonthDay(df)

In [ ]:
PriceData = df

In [ ]:
PriceData.jalaliDate.max()

In [ ]:
'
df2 = pd.read_csv(n2)
df2 = df2[['symbol', 'year','total_assets']].rename(columns = {"year":'Year',"total_assets":'BookValue'})
df2['Year'] = df2['Year'].astype(str)
PriceData = PriceData.merge(df2,on=['symbol','Year']).drop(columns = ['Unnamed: 0'])

In [ ]:
PriceData.columns

In [ ]:
len(set(WeeklyHolderData.symbol))

In [ ]:
WinLos = PriceData[['jalaliDate', 'date', 'symbol','close_price']]
WinLos['date1'] = WinLos['date'].apply(vv4)
WinLos['date1'] = pd.to_datetime(WinLos['date1']) 
WinLos['week_of_year'] = WinLos['date1'].dt.week 
WinLos['year_of_year'] = WinLos['date1'].dt.year

In [ ]:
def weekRet(g):
    g['weekRet'] = (g.close_price.iloc[-1] - g.close_price.iloc[0])/g.close_price.iloc[0] * 100
    return g

gg = WinLos.groupby(['symbol', 'year_of_year','week_of_year'])
d = gg.apply(weekRet)
#Don't Delete this cell

In [ ]:
def yearRet(g):
    f = g.groupby(['year_of_year']).last().reset_index()
    f['yearRet'] = f['close_price'].pct_change(periods = 1)*100
    f = f[['year_of_year','symbol','week_of_year','yearRet']]
    g = g.merge(f,on = ['year_of_year','symbol','week_of_year'])
    return g

gg = d.groupby(['symbol', 'week_of_year'])
d2 = gg.apply(yearRet)

In [ ]:
WinLosData = d2.reset_index(drop = True)

In [ ]:
WinLosData = WinLosData.groupby(['symbol','year_of_year','week_of_year']).last().reset_index()

In [ ]:
WinLosData = WinLosData[WinLosData.jalaliDate > 13960000].dropna()

In [ ]:
WinLosData = WinLosData.groupby(['year_of_year','week_of_year']).apply(lambda x: x.sort_values(["yearRet"], ascending = False)).reset_index(drop=True)

In [ ]:
def WinLos(g):
    Winner = g.loc[g['yearRet'] >= g['yearRet'].quantile(0.9)]['yearRet'].mean()
    Loser = g.loc[g['yearRet'] <= g['yearRet'].quantile(0.1)]['yearRet'].mean()
    g['Winner_Loser'] = Winner - Loser
    g = g[['year_of_year','week_of_year','Winner_Loser']].drop_duplicates()
    return g

WL = WinLosData.groupby(['year_of_year','week_of_year'])
WinLosData = WL.apply(WinLos)

In [ ]:
WinLosData

In [ ]:
WinLosData = WinLosData.reset_index(drop = True)

In [ ]:
PriceData.jalaliDate.max()

In [ ]:
WinLosData

In [ ]:
shrout = HolderData[['symbol','shrout','date','jalaliDate']].drop_duplicates()
PriceData = PriceData.merge(shrout,on = ['symbol','date','jalaliDate'])

In [ ]:
PriceData['date1'] = PriceData['date'].apply(vv4)
PriceData['date1'] = pd.to_datetime(PriceData['date1']) 
PriceData['week_of_year'] = PriceData['date1'].dt.week 
PriceData['year_of_year'] = PriceData['date1'].dt.year

In [ ]:
mapingdict = dict(zip(zip(list(WinLosData.year_of_year),list(WinLosData.week_of_year)),list(WinLosData.Winner_Loser)))
PriceData['Winner_Loser'] = PriceData.set_index(['year_of_year','week_of_year']).index.map(mapingdict)

In [ ]:
PriceData = PriceData[['jalaliDate', 'date', 'symbol','group_name','close_price','BookValue', 'shrout','week_of_year', 'year_of_year','Winner_Loser']]

In [ ]:
PriceData.head()

In [ ]:
path2 = r"G:\TseClient\Data adjusted"
index = pd.read_excel(path2 + '\IRX6XTPI0009.xls')[['<COL14>','<CLOSE>']].rename(columns = {'<COL14>':'jalaliDate','<CLOSE>':'Index'})
index['Market_return'] =  index['Index'].pct_change(periods = 1)*100
# index = index.drop(columns = ['Index'])

In [ ]:
PriceData = PriceData.merge(index,on = ['jalaliDate']).drop_duplicates()

In [ ]:
d = d.drop(columns = ['close_price','date1'])

In [ ]:
WeekPriceData = PriceData.groupby(['symbol','year_of_year','week_of_year']).last().reset_index()
WeekPriceData = WeekPriceData.merge(d,on = ['symbol','year_of_year','week_of_year','jalaliDate','date'])

In [ ]:
WeekPriceData['MarketCap']= WeekPriceData['close_price'] * WeekPriceData['shrout']/1e5
WeekPriceData['MarketCap'] = WeekPriceData['MarketCap'].astype(int)
WeekPriceData['BookToMarket'] = WeekPriceData['BookValue']/WeekPriceData['MarketCap']

In [ ]:
Index = PriceData[['week_of_year','year_of_year','Winner_Loser','Index','Market_return']].drop_duplicates()

In [ ]:

def weekRet(g):
    g['MarketweekRet'] = (g.Index.iloc[-1] - g.Index.iloc[0])/g.Index.iloc[0] * 100
    return g

gg = Index.groupby(['year_of_year','week_of_year'])
WeekIndex = gg.apply(weekRet)
#Don't Delete this cell


In [ ]:
# WeekIndex = 
WeekIndex = WeekIndex.groupby(['year_of_year','week_of_year']).last().reset_index()[['year_of_year','week_of_year','MarketweekRet']]

In [ ]:
WeekIndex.head()

In [ ]:
def SMB(g):
    Large = g.loc[g['MarketCap'] >= g['MarketCap'].quantile(0.9)]['weekRet'].mean()
    Small = g.loc[g['MarketCap'] <= g['MarketCap'].quantile(0.1)]['weekRet'].mean()
    Value = g.loc[g['BookToMarket'] >= g['BookToMarket'].quantile(0.9)]['weekRet'].mean()
    Growth = g.loc[g['BookToMarket'] <= g['BookToMarket'].quantile(0.1)]['weekRet'].mean()
    
    g['SMB'] = Small - Large
    g['HML'] = Value - Growth
    
    g = g[['year_of_year','week_of_year','SMB','HML','Winner_Loser']].drop_duplicates(['year_of_year','week_of_year'])
    return g

WL = WeekPriceData.groupby(['year_of_year','week_of_year'])
Factors = WL.apply(SMB)
Factors = Factors.reset_index(drop = True)


In [ ]:
mapingdict = dict(zip(zip(list(WeekIndex.year_of_year),list(WeekIndex.week_of_year)),list(WeekIndex['MarketweekRet'])))
Factors['MarketweekRet'] = Factors.set_index(['year_of_year','week_of_year']).index.map(mapingdict)

In [ ]:
Factors.to_excel(path + '\Analyzed Data'+'\Factors.xlsx',index= False)

In [ ]:
Factors = pd.read_excel(path + '\Analyzed Data'+'\Factors.xlsx')

In [ ]:
Factors

In [ ]:
HolderData
HolderData['date1'] = HolderData['date'].apply(vv4)
HolderData['date1'] = pd.to_datetime(HolderData['date1']) 
HolderData['week_of_year'] = HolderData['date1'].dt.week 
HolderData['Month_of_year'] = HolderData['date1'].dt.month 
HolderData['year_of_year'] = HolderData['date1'].dt.year

In [ ]:
ldates = HolderData.groupby(['symbol','year_of_year','Month_of_year','week_of_year']).last()
ldates = ldates.reset_index()[['symbol','date']]


In [ ]:
[1]*5

In [ ]:
fkey = zip(list(ldates.symbol),list(ldates.date))
mapingdict = dict(zip(fkey,[1]*(len(ldates.symbol)+1)))
HolderData['Last'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)

In [ ]:
WeeklyHolderData = HolderData.loc[HolderData.Last == 1]

In [ ]:
for t in ['SMB','HML','Winner_Loser','MarketweekRet']: 
    print(t)
    mapingdict = dict(zip(zip(list(Factors.year_of_year),list(Factors.week_of_year)),list(Factors[t])))
    WeeklyHolderData[t] = WeeklyHolderData.set_index(['year_of_year','week_of_year']).index.map(mapingdict)

In [ ]:
fkey = zip(list(d.symbol),list(d.date))
mapingdict = dict(zip(fkey,d.weekRet))
WeeklyHolderData['weekRet'] = WeeklyHolderData.set_index(['symbol','date']).index.map(mapingdict)

In [ ]:
WeekPriceData = WeeklyHolderData.groupby(['symbol','year_of_year','Month_of_year','week_of_year']).last().reset_index()

In [ ]:
WeekPriceData = WeekPriceData.dropna()

In [ ]:
wg = WeekPriceData.groupby(['symbol'])
g = wg.get_group('آ س پ')
g
# y = 'weekRet'
# x = ['MarketweekRet','HML','SMB','Winner_Loser']
# model = sm.OLS(g[y], sm.add_constant(g[x])).fit()

In [ ]:
wg = WeekPriceData.groupby(['symbol'])
def FourFactor(g):
    try:
        y = 'weekRet'
        x = ['MarketweekRet','HML','SMB','Winner_Loser']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['4-Residual'] = g ['weekRet'] - (model.params[0] + model.params[1]*g['MarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def ThreeFactor(g):
    try:
        y = 'weekRet'
        x = ['MarketweekRet','HML','SMB']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['3-Residual'] = g ['weekRet'] - (model.params[0] + model.params[1]*g['MarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB'] ) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g
re = wg.apply(FourFactor)
re2 = wg.apply(ThreeFactor)
re['3-Residual'] = re2['3-Residual']
re = re.reset_index(drop=True).drop(columns = ['group_name'])

In [ ]:
df = HolderData 

In [ ]:
df= df[df.Trade == 'Yes']

In [ ]:
df['date1'] = df['date'].apply(vv4)
df['date1'] = pd.to_datetime(df['date1']) 
df['week_of_year'] = df['date1'].dt.week 
df['year_of_year'] = df['date1'].dt.year

In [ ]:
df = df.merge(re,on=['symbol', 'year_of_year', 'week_of_year', 'jalaliDate', 'date','close_price','shrout'])

In [ ]:
df.head()

In [ ]:
Holders = list(set(df.Holder))
Holders.sort()
ids = list(range(len(Holders)))
mapingdict = dict(zip(Holders, ids))
df['Holder_id'] = df['Holder'].map(mapingdict)

symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(symbols, ids))
df['id'] = df['symbol'].map(mapingdict)

In [ ]:
df['month_of_year'] = df['date1'].dt.month 

In [ ]:
gdata = df.groupby(['id'])
g = gdata.get_group(2)

In [ ]:
data = pd.DataFrame()
gg = df.groupby(['id'])

def FCAPf(S_g,g):
    intersection = list(set.intersection(set(S_g.date), set(g.date)))
    g = g.loc[g.date.isin(intersection)]
    S_g = S_g.loc[S_g.date.isin(intersection)]
    a = g.merge(S_g , on = ['Holder_id','date'])
    if len(a) == 0: 
        return
    else:
        intersection = list(set.intersection(set(S_g.date), set(g.date)))
        g = g.loc[g.date.isin(intersection)]
        S_g = S_g.loc[S_g.date.isin(intersection)]
        a = g.merge(S_g , on = ['Holder_id','date','jalaliDate','week_of_year','month_of_year','year_of_year'])
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['SizeRatio'] = (a['MarketCap_x'])/(a['MarketCap_y'])

        f = a.groupby(['date','jalaliDate','id_x','id_y','week_of_year','month_of_year','year_of_year','group_name_x','group_name_y','weekRet_x','weekRet_y','SizeRatio','BookToMarket_x','BookToMarket_y','MarketCap_x','MarketCap_y','4-Residual_x','4-Residual_y','3-Residual_x','3-Residual_y'])['FCAPf'].sum().to_frame().reset_index()
        fc = f.groupby(['year_of_year','month_of_year'])[['4-Residual_x','4-Residual_y','3-Residual_x','3-Residual_y']].corr().reset_index()
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','month_of_year','4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '3-Residual_y'][['year_of_year','month_of_year','3-Residual_x']].rename(columns = {'3-Residual_x':'ρ_3'})
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['ρ_4'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_3)))
        f['ρ_3'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)  
        f['FCAPF'] = (f['FCAPf'] - f['FCAPf'].mean()).divide(f['FCAPf'].std())
        return f

for i in gdata.groups.keys():
    g = gdata.get_group(i)
    F_id = g.id.iloc[0]
    print(F_id)
    Next_df = df[df.id > F_id]
    S_gg = Next_df.groupby(['id'])
    data = data.append(S_gg.apply(FCAPf , g = g ))

In [ ]:
d = data.reset_index(drop = True).drop_duplicates(['id_x','id_y','month_of_year','year_of_year']).reset_index(drop = True)

In [ ]:
symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(ids,symbols))
d['symbol_x'] = d['id_x'].map(mapingdict)
d['symbol_y'] = d['id_y'].map(mapingdict)
d

In [ ]:
d['month_of_year'] = d['month_of_year'].astype(str)
def add(row):
    if len(row)<2:
        row = '0'+ row
    return row

d['month_of_year'] = d['month_of_year'].apply(add )

In [ ]:
d['year_of_year'] = d['year_of_year'].astype(str)

d['Year_Month'] = d['year_of_year'] + d['month_of_year']
d

In [ ]:
days = list(set(d.Year_Month))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t'] = d['Year_Month'].map(mapingdict)

In [ ]:
d

In [ ]:
d['id_x'] = d['id_x'].astype(str)
d['id_y'] = d['id_y'].astype(str)
d['id'] = d['symbol_x'] + d['symbol_y']
ids = list(set(d.id))
ids.sort()
id = list(range(len(ids)))
mapingdict = dict(zip(ids, id))
d['id'] = d['id'].map(mapingdict)

In [ ]:
d.head()

In [ ]:
d['size1'] = 0
d.loc[d.MarketCap_x>d.MarketCap_y,'size1'] = d[d.MarketCap_x>d.MarketCap_y]['MarketCap_x']
d.loc[d.MarketCap_x<d.MarketCap_y,'size1'] = d[d.MarketCap_x<d.MarketCap_y]['MarketCap_y']
d['size2'] = 0
d.loc[d.MarketCap_x>d.MarketCap_y,'size2'] = d[d.MarketCap_x>d.MarketCap_y]['MarketCap_y']
d.loc[d.MarketCap_x<d.MarketCap_y,'size2'] = d[d.MarketCap_x<d.MarketCap_y]['MarketCap_x']


In [ ]:
d.head()

In [ ]:
d['ρ3_f'] = d.groupby(['id'])['ρ_3'].shift(-1)
d['ρ4_f'] = d.groupby(['id'])['ρ_4'].shift(-1)
d['ρ3_f2'] = d.groupby(['id'])['ρ_3'].shift(-2)
d['ρ4_f2'] = d.groupby(['id'])['ρ_4'].shift(-2)


In [ ]:
d['SameSize'] = d['size1'] - d['size2']
d['SameSize'] = (d['SameSize']-d['SameSize'].mean()).divide(d['SameSize'].std())
d

In [ ]:
d['sgroup'] = 0
d.loc[d.group_name_x == d.group_name_y,'sgroup'] = 1

In [ ]:
n3 = path + '\\NormalzedFCAP2.csv'
d.to_csv(n3,index = False)

In [ ]:
path

In [ ]:
d[d.id == 4006]

In [ ]:
d[d.duplicated(['t','id'])]